Imports

In [ ]:
# home-robot imports
import argparse
import json
import os
import sys
import time
from collections import defaultdict
from pathlib import Path

import numpy as np
from config_utils import get_config
from omegaconf import DictConfig, OmegaConf


from habitat import make_dataset
from habitat.core.environments import get_env_class
from habitat.core.vector_env import VectorEnv
from habitat.core.env import Env
from habitat.utils.gym_definitions import _get_env_name
from habitat_baselines.rl.ppo.ppo_trainer import PPOTrainer

from home_robot.agent.ovmm_agent.ovmm_agent import OpenVocabManipAgent
from home_robot_sim.env.habitat_ovmm_env.habitat_ovmm_env import (
    HabitatOpenVocabManipEnv,
)

All code should be ran from the root home-robot directory

In [ ]:
os.chdir("../..") # change directory to $HOME_ROBOT_ROOT
#os.chdir(os.environ['HOME_ROBOT_ROOT'])

In [ ]:
%pwd

Setup config 

In [ ]:
config_path = "ovmm/ovmm_eval.yaml"
config, config_str = get_config(config_path)
OmegaConf.set_readonly(config, False)

config.habitat_baselines.num_environments = 1
OmegaConf.set_struct(config.habitat_baselines, False)
config.habitat_baselines.print_images = 1
config.habitat.dataset.split = "val"
config.habitat_baselines.exp_name = "debug_notebook"

OmegaConf.set_readonly(config, True)
baseline_config = OmegaConf.load(
    "projects/habitat_ovmm/configs/agent/hssd_eval.yaml"
)
config = DictConfig({**config, **baseline_config})
config.VISUALIZE = 1
print(config)

In [ ]:
habitat_config = config.habitat
dataset = make_dataset(habitat_config.dataset.type, config=habitat_config.dataset)
env_class_name = _get_env_name(config)
env_class = get_env_class(env_class_name)

Create the agent and environment

In [ ]:
agent = OpenVocabManipAgent(config=config)

habitat_env = env_class(config=habitat_config, dataset=dataset) # TODO: This line stalls on first execution. 
                                                                # current workaround is to interrupt after a few seconds and rerun
habitat_env.seed(habitat_config.seed)
env = HabitatOpenVocabManipEnv(habitat_env, config, dataset=dataset)

In [ ]:
t = 0
env.reset()
agent.reset()

In [ ]:
t += 1
obs = env.get_observation()
action, info, act_obs = agent.act(obs)
print(t, action)
env.apply_action(action, info)

In [ ]:
#Step all the way
#while not env.episode_over:
for _ in range(100):
    t += 1
    obs = env.get_observation()
    action, info, act_obs = agent.act(obs)
    print(t, action)
    env.apply_action(action, info)

print(env.get_episode_metrics())